In [1]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/Dominik/R-workspace/cecelia/inst')

# MacOS
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [2]:
base_dir = '/Volumes/Analysis_SSD/Dominik/cecelia/projects/8BR53W/ANALYSIS/'
zero_dir = os.path.join(base_dir, '0/yBPJeU/')
#im_path = os.path.join(zero_dir, 'ccidImage.ome.zarr')
im_path = os.path.join(zero_dir, 'ccidCorrected.zarr')
#im_path = os.path.join(zero_dir, 'ccidSlidingWindow.zarr')
version_num = 1
task_dir = os.path.join(base_dir, str(version_num), 'yBPJeU')

In [3]:
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.open_viewer()
napari_utils.task_dir = task_dir

/Users/Dominik/R-workspace/cecelia/inst/py/shape_utils.py:6: FutureWarning: csv_to_layer_data was moved from napari.utils.io in v0.4.17. Import it from napari_builtins.io instead.
  from napari.utils.io import csv_to_layer_data
napari.manifest -> 'skan' could not be imported: Could not find file 'napari.yaml' in module 'skan'


In [4]:
channel_names = [
    'EB', 'T-A', 'T-B', 'AF', 'gen'
    #'One'
]

In [5]:
napari_utils.open_image(
    im_path,
    use_channel_axis = True, as_dask = True,
    downsample_z = False, channel_names = channel_names,
    show_3D = True, visible = [True, True, True, True, False],
    channel_colormaps = ['gray', None, None, 'red', None]
)

[90, 4, 22, 520, 514]
[514, 520, 22, 4, 90]
['X', 'Y', 'Z', 'C', 'T']
[90, 4, 22, 520, 514]


In [6]:
value_name = 'OTI'

In [ ]:
prop_df = napari_utils.label_props_utils.label_props_view(value_name = value_name)\
    .exclude_centroid_cols()\
    .exclude_obs_cols(['label'])\
    .as_df()\
    .dropna(subset=['track_id'])\
    .replace(False, 0)\
    .replace(True, 1)

In [ ]:
prop_df

In [ ]:
i = 'live.cell.track.clusters.hmm_transitions_movement'

In [ ]:
prop_df[i]

In [ ]:
import pandas as pd
cat_columns = prop_df.select_dtypes(['category']).columns
prop_df[cat_columns] = prop_df[cat_columns].apply(
    lambda x: pd.to_numeric(x, errors = 'coerce'))

In [ ]:
np.unique(prop_df[i].isin([2]).values)

In [ ]:
napari_utils.dim_utils.im_dim

In [ ]:
napari_utils.viewer.window.resize(1300,1000)
napari_utils.viewer.reset_view()

In [9]:
napari_utils.show_labels_all(
    show_labels = False,
    show_points = False,
    show_tracks = True,
    value_names = ['OTI'],
    split_tracks = {
        'live.cell.track.clusters.hmm_transitions_movement': {
            'others': {'values': [0,1,3,4,6], 'colour': '#B3BCC2'},
            'scanning': {'values': [5], 'colour': '#4682b4'},
            'directed': {'values': [2], 'colour': '#AA1F5E'}
        }
    }
)

Index(['live.cell.hmm.transitions.movement',
       'live.cell.track.clusters.hmm_transitions_movement',
       'live.cell.track.clusters.hmm_movement'],
      dtype='object')


/Users/Dominik/Library/r-miniconda/envs/r-cecelia-env/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:623: UserWarning: Previous color_by key 'live.cell.track.clusters.hmm_transitions_movement' not present in features. Falling back to track_id
  warn(
/Users/Dominik/Library/r-miniconda/envs/r-cecelia-env/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:623: UserWarning: Previous color_by key 'live.cell.track.clusters.hmm_transitions_movement' not present in features. Falling back to track_id
  warn(
/Users/Dominik/Library/r-miniconda/envs/r-cecelia-env/lib/python3.9/site-packages/napari/layers/tracks/tracks.py:623: UserWarning: Previous color_by key 'live.cell.track.clusters.hmm_transitions_movement' not present in features. Falling back to track_id
  warn(


In [8]:
napari_utils.show_labels_all(
    show_labels = False,
    show_points = False,
    show_tracks = True,
    value_names = ['OTI']
)

Index(['live.cell.hmm.transitions.movement',
       'live.cell.track.clusters.hmm_transitions_movement',
       'live.cell.track.clusters.hmm_movement'],
      dtype='object')


In [ ]:
# helper function to dispaly the animation in the notebook
from IPython.display import Video, display


def jupyter_napari_animation(animation, **kwargs):
    from pathlib import Path
    from datetime import datetime

    dir = Path.cwd() / "media"
    dir.mkdir(exist_ok=True)
    timestamp = datetime.now().strftime("%Y-%m-%d@%H-%M-%S")
    mov_name = "napari_video" + "@" + timestamp + ".mp4"
    path = dir / mov_name
    animation.animate(path, **kwargs)
    vid = Video(path, width=600, html_attributes=f"controls autoplay loop")
    display(vid)



In [ ]:
from napari_animation import Animation
from napari_animation.easing import Easing

In [ ]:
movies_path = '/Users/Dominik/Downloads/movies'

In [ ]:
animation = Animation(napari_utils.viewer)

# first
dim_array = np.zeros_like(napari_utils.dim_utils.dim_vals(ignore_channel = True))
t_val = napari_utils.dim_utils.dim_val('T')

napari_utils.viewer.dims.current_step = list(dim_array)
animation.capture_keyframe(steps = 0)

# last
dim_array[napari_utils.dim_utils.dim_idx('T', ignore_channel = True)] = t_val - 1

napari_utils.viewer.dims.current_step = list(dim_array)
animation.capture_keyframe(steps = t_val - 2)

animation.animate(movies_path, canvas_only = True, fps = 10)

In [ ]:
# show populations
napari_utils.show_pop_mapping('live')

In [ ]:
# save image as tiff
import py.tiff_utils as tiff_utils

tiff_utils.save_as_tiff(
    '/Volumes/USER_data/Dominik/Experiments/TEAGEN_SKIN/DATA/PuJo5p.tif',
    napari_utils.im_data[0], napari_utils.dim_utils, imagej = True)

In [ ]:
print('here')

In [ ]:
napari_utils.im_scale

In [ ]:
from napari_animation import AnimationWidget
animation_widget = AnimationWidget(napari_utils.viewer)
napari_utils.viewer.window.add_dock_widget(animation_widget, area='right')

In [ ]:
pop_type = 'live'
pop_map = napari_utils.pop_utils.pop_map(napari_utils.task_dir, pop_type)
pop_data = napari_utils.pop_utils.pop_data(napari_utils.task_dir, pop_type)

In [ ]:
pop_info = pop_map['5iaWSo']

In [ ]:
pop_info